# 0. Preparing
Install ortools package

In [1]:
%pip install ortools

# 1. Import modules and load the data set

In [2]:
from ortools.sat.python import cp_model
from google.colab import files
import pandas as pd
import io, os

fname = "학급반편성CSP 문제 입력파일.csv"
sep = ","

if os.path.exists(fname):
  os.remove(fname)
uploaded = files.upload()

df = pd.read_csv(io.StringIO(
 uploaded[fname].decode('utf-8')), sep=sep)


Saving 학급반편성CSP 문제 입력파일.csv to 학급반편성CSP 문제 입력파일.csv


#2. Define the CP model and variables

In [3]:
model = cp_model.CpModel()

NUM_STUDENTS = len(df)  # Number of students (is 200)
NUM_CLASSROOMS = 6      # Number of classrooms
CLASSROOM_IDS = list(range(NUM_CLASSROOMS))   # Id of the classrooms (0 .. 5)
CLASSROOM_SIZES = {0: 33, 1: 33, 2: 33, 3: 33, 4: 34, 5: 34}    # Count of students of each classroom

# int : pd.Series
students_dict = {row["id"]: row for _, row in df.iterrows()}

student_vars = {}
for s_id in students_dict.keys():
    student_vars[s_id] = model.NewIntVar(
        0, NUM_CLASSROOMS - 1, f"student_{s_id}_class"
    )

# 3. Set the constraint

In [4]:
# Const 0. Students must be assigned filling the size of classrooms
assigned_to_class = {}

for s_id in students_dict.keys():
    assigned_to_class[s_id] = {}

    for c_id in CLASSROOM_IDS:
        assigned_to_class[s_id][c_id] = model.NewBoolVar(f"student_{s_id}_in_class_{c_id}")
        # Link student_vars to these boolean variables:
        model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(assigned_to_class[s_id][c_id])
        model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(assigned_to_class[s_id][c_id].Not())

for c_id in CLASSROOM_IDS:
    students_in_this_class = [assigned_to_class[s_id][c_id] for s_id in students_dict.keys()]
    model.Add(sum(students_in_this_class) == CLASSROOM_SIZES[c_id])

In [5]:
# Const 1-A. Students who have bad relationship must be seperated
for sid, row in students_dict.items():
    if pd.notna(row["나쁜관계"]):
        bad_id = str(row["나쁜관계"]).strip()
        if bad_id: model.Add(student_vars[sid] != student_vars[int(float(bad_id))])

# Const 1-B. Students who have good relationship must be in the same class
for sid, row in students_dict.items():
    if pd.notna(row["좋은관계"]):
        good_id = str(row["좋은관계"]).strip()
        if good_id: model.Add(student_vars[sid] == student_vars[int(float(good_id))])

In [6]:
# Const 2. At least one student who has leadership should be assigned to each class
leader_assigned_to_class = {}

for s_id in students_dict.keys():
    if str(students_dict[s_id]["Leadership"]) == "yes":
        leader_assigned_to_class[s_id] = {}
    else: continue

    for c_id in CLASSROOM_IDS:
        leader_assigned_to_class[s_id][c_id] = model.NewBoolVar(f"leader_{s_id}_in_class_{c_id}")
        model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(leader_assigned_to_class[s_id][c_id])
        model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(leader_assigned_to_class[s_id][c_id].Not())

for c_id in CLASSROOM_IDS:
    model.Add(sum(leader_assigned_to_class[s_id][c_id] for c_id in CLASSROOM_IDS) > 0)

In [7]:
# Const 3. Students who can play piano must be distributed equally
piano_assigned_to_class = {}

for s_id in students_dict.keys():
    if str(students_dict[s_id]["Piano"]) == "yes":
        piano_assigned_to_class[s_id] = {}
    else: continue

    for c_id in CLASSROOM_IDS:
        piano_assigned_to_class[s_id][c_id] = model.NewBoolVar(f"piano_{s_id}_in_class_{c_id}")
        model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(piano_assigned_to_class[s_id][c_id])
        model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(piano_assigned_to_class[s_id][c_id].Not())

min_piano_students_per_class = len(piano_assigned_to_class) // NUM_CLASSROOMS - 1
max_piano_students_per_class = len(piano_assigned_to_class) // NUM_CLASSROOMS + 1
if min_piano_students_per_class < 1: min_piano_students_per_class = 1

for c_id in CLASSROOM_IDS:
    model.Add(sum(piano_assigned_to_class[s_id][c_id] for s_id in piano_assigned_to_class) >= min_piano_students_per_class)
    model.Add(sum(piano_assigned_to_class[s_id][c_id] for s_id in piano_assigned_to_class) <= max_piano_students_per_class)


# Const 4. Minimize the score deviation between the classes...?
#          This seems to be an optimization objective


In [8]:
# Const 5. Abscent students should be distributed equally
abscence_assigned_to_class = {}

for s_id in students_dict.keys():
    if str(students_dict[s_id]["비등교"]) == "yes":
        abscence_assigned_to_class[s_id] = {}
    else: continue

    for c_id in CLASSROOM_IDS:
        abscence_assigned_to_class[s_id][c_id] = model.NewBoolVar(f"abscence_{s_id}_in_class_{c_id}")
        model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(abscence_assigned_to_class[s_id][c_id])
        model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(abscence_assigned_to_class[s_id][c_id].Not())

min_abscence_students_per_class = len(abscence_assigned_to_class) // NUM_CLASSROOMS - 1
max_abscence_students_per_class = len(abscence_assigned_to_class) // NUM_CLASSROOMS + 1

for c_id in CLASSROOM_IDS:
    model.Add(sum(abscence_assigned_to_class[s_id][c_id] for s_id in abscence_assigned_to_class) >= min_abscence_students_per_class)
    model.Add(sum(abscence_assigned_to_class[s_id][c_id] for s_id in abscence_assigned_to_class) <= max_abscence_students_per_class)

In [9]:
# Const 6. Equalize the sex ratio
balance_vars = []
for c_id in CLASSROOM_IDS:
    boy_count = int(dict(df['sex'].value_counts())['boy'])
    girl_count = NUM_STUDENTS - boy_count  # no such sex like attack heli


    diff = model.NewIntVar(-NUM_STUDENTS, NUM_STUDENTS, f"sex_diff_{c_id}")
    abs_diff = model.NewIntVar(0, NUM_STUDENTS, f"abs_sex_diff_{c_id}")
    model.AddAbsEquality(abs_diff, diff)
    model.add(abs_diff <= 1)

In [10]:
# Const 7. Student who has good physical ability...
phys_assigned_to_class = {}

for s_id in students_dict.keys():
    if str(students_dict[s_id]["운동선호"]) == "yes":
        phys_assigned_to_class[s_id] = {}
    else: continue

    for c_id in CLASSROOM_IDS:
        phys_assigned_to_class[s_id][c_id] = model.NewBoolVar(f"phys_{s_id}_in_class_{c_id}")
        model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(phys_assigned_to_class[s_id][c_id])
        model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(phys_assigned_to_class[s_id][c_id].Not())

min_phys_students_per_class = len(phys_assigned_to_class) // NUM_CLASSROOMS - 1
max_phys_students_per_class = len(phys_assigned_to_class) // NUM_CLASSROOMS + 1

for c_id in CLASSROOM_IDS:
    model.Add(sum(phys_assigned_to_class[s_id][c_id] for s_id in phys_assigned_to_class) >= min_phys_students_per_class)
    model.Add(sum(phys_assigned_to_class[s_id][c_id] for s_id in phys_assigned_to_class) <= max_phys_students_per_class)

In [11]:
# Const 8. Students who were in the same class should be seperated
from collections import Counter
prev_class_counts = Counter(
    [str(s["24년 학급"]).strip()
    for s in students_dict.values()
    if pd.notna(s["24년 학급"])]
)

prev_class_dict = {}
for c in 'abcdef':
    prev_class_dict[c] = {}
    for s_id in students_dict.keys():
        if str(students_dict[s_id]["24년 학급"]).strip() == c:
            prev_class_dict[c][s_id] = {}
        else: continue

        for c_id in CLASSROOM_IDS:
            prev_class_dict[c][s_id][c_id] = model.NewBoolVar(f"{s_id}_prev_class_{c}_in_class_{c_id}")
            model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(prev_class_dict[c][s_id][c_id])
            model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(prev_class_dict[c][s_id][c_id].Not())

    min_prev_class_students_per_class = prev_class_counts[c] // NUM_CLASSROOMS - 1
    max_prev_class_students_per_class = prev_class_counts[c] // NUM_CLASSROOMS + 1

    for c_id in CLASSROOM_IDS:
        model.Add(sum(prev_class_dict[c][s_id][c_id] for s_id in prev_class_dict[c]) >= min_prev_class_students_per_class)
        model.Add(sum(prev_class_dict[c][s_id][c_id] for s_id in prev_class_dict[c]) <= max_prev_class_students_per_class)

In [12]:
# Const 9. Club members...
club_counts = Counter(
    [str(s["클럽"]).strip()
    for s in students_dict.values()
    if pd.notna(s["클럽"])]
)

club_dict = {}
for c in sorted(club_counts):
    club_dict[c] = {}
    for s_id in students_dict.keys():
        if str(students_dict[s_id]["클럽"]).strip() == c:
            club_dict[c][s_id] = {}
        else: continue

        for c_id in CLASSROOM_IDS:
            club_dict[c][s_id][c_id] = model.NewBoolVar(f"{s_id}_club_{c}_in_class_{c_id}")
            model.Add(student_vars[s_id] == c_id).OnlyEnforceIf(club_dict[c][s_id][c_id])
            model.Add(student_vars[s_id] != c_id).OnlyEnforceIf(club_dict[c][s_id][c_id].Not())

    min_club_students_per_class = club_counts[c] // NUM_CLASSROOMS - 1
    max_club_students_per_class = club_counts[c] // NUM_CLASSROOMS + 1

    for c_id in CLASSROOM_IDS:
        model.Add(sum(club_dict[c][s_id][c_id] for s_id in club_dict[c]) >= min_club_students_per_class)
        model.Add(sum(club_dict[c][s_id][c_id] for s_id in club_dict[c]) <= max_club_students_per_class)

# 4. Satisfy the optimization objective

In [13]:
student_scores = {row["id"]: row["score"] for _, row in df.iterrows()}

# Calculate the total score for each class
classroom_total_scores = {}
for c_id in CLASSROOM_IDS:
    sum_expr = sum(assigned_to_class[s_id][c_id] * student_scores[s_id]
                   for s_id in students_dict)
    classroom_total_scores[c_id] = model.NewIntVar(0, sum(student_scores.values()), f"class_{c_id}_total_score")
    model.Add(classroom_total_scores[c_id] == sum_expr)

min_score = model.NewIntVar(0, sum(student_scores.values()), "min_score")
max_score = model.NewIntVar(0, sum(student_scores.values()), "max_score")

model.AddMinEquality(min_score, [classroom_total_scores[c_id] for c_id in CLASSROOM_IDS])
model.AddMaxEquality(max_score, [classroom_total_scores[c_id] for c_id in CLASSROOM_IDS])


model.Minimize(max_score-min_score)

# 5. Solve the model

In [14]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60
solver.parameters.num_search_workers = 8

status = solver.Solve(model)

#6. Display the results

In [15]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

    print(f"\nSolution found (Status: {solver.StatusName(status)})")
    print(f"  - Objective value (Min/Max score difference): {solver.ObjectiveValue()}")
    print(f"  - Wall time: {solver.WallTime():.2f}s")
    #print(f"  - Solve time: {solver.ProtoStats().solve_time_in_seconds:.2f}s")
    #'CpSolver' object has no attribute 'ProtoStats'


    total_overall_score = sum(student_scores.values())
    print(f"\nTotal Students: {NUM_STUDENTS}, Total Classrooms: {NUM_CLASSROOMS}")
    print(f"Overall Average Student Score: {total_overall_score / NUM_STUDENTS:.4f}")
    print(f"Ideal Students per Class: {NUM_STUDENTS / NUM_CLASSROOMS:.2f}")
    print(f"Ideal Total Score per Class: {total_overall_score / NUM_CLASSROOMS:.4f}\n")

    classroom_assignments = {c_id: [] for c_id in CLASSROOM_IDS}
    for student_id in students_dict.keys():
        assigned_class = solver.Value(student_vars[student_id])
        classroom_assignments[assigned_class].append(student_id)

    # Display the result of Assignment
    new_class_df = pd.DataFrame(columns=['id', 'class'])
    for c_id in CLASSROOM_IDS:
        students_in_class = classroom_assignments[c_id]
        for student_id in students_in_class:
            new_class_df = pd.concat([new_class_df, pd.DataFrame({'id': [student_id], 'class': [c_id+1]})], ignore_index=True)
    print(f"\n\n:::Result:::\n{new_class_df}\n\n")


    classrooms = {c_id: f"class_{c_id}" for c_id in CLASSROOM_IDS}


    for c_id in CLASSROOM_IDS:
        students_in_class = classroom_assignments[c_id]
        class_scores = [student_scores[s_id] for s_id in students_in_class]
        num_students = len(students_in_class)
        class_total_score = sum(class_scores)
        class_avg_score = class_total_score / num_students if num_students > 0 else 0

        leadership_student = [s_id for s_id in students_in_class if students_dict[s_id]["Leadership"] == "yes"]

        club_students_dict = {str(s["클럽"]).strip(): [] for _, s in df.iterrows()}
        for s_id in students_in_class:
            club_students_dict[str(students_dict[s_id]["클럽"]).strip()].append(s_id)
        for club in club_students_dict:
            club_students_dict[club] = len(club_students_dict[club])

        prev_class_students_dict = {str(s["24년 학급"]).strip(): [] for _, s in df.iterrows()}
        for s_id in students_in_class:
            prev_class_students_dict[str(students_dict[s_id]["24년 학급"]).strip()].append(s_id)
        for prev_class in prev_class_students_dict:
            prev_class_students_dict[prev_class] = len(prev_class_students_dict[prev_class])

        male_count = sum(1 for s_id in students_in_class if students_dict[s_id]["sex"] == "boy")
        female_count = num_students - male_count



        print(f"-------- Classroom {c_id} --------")
        print(f"  Number of Students: {num_students}")
        print(f"  Total Score: {class_total_score}")
        print(f"  Average Score: {class_avg_score:.2f}")
        print(f"  Leadership students: {leadership_student}")
        print(f"  Count of club members: {club_students_dict}")
        print(f"  Count of prev classes: {prev_class_students_dict}")
        print(f"  Sex ratio: M {male_count} : {female_count} F")



        # Verify dislike constraint for this class
        dislike_violations = []

        # Reconstruct disliked pairs from the dataframe for verification
        current_class_student_ids = set(students_in_class)
        for sid, row in students_dict.items():
            if sid in current_class_student_ids and pd.notna(row["나쁜관계"]) and str(row["나쁜관계"]).strip() != "":
                 bad_list = str(row["나쁜관계"]).split(",")
                 for bad_id_str in bad_list:
                     bad_id_str = bad_id_str.strip()
                     if bad_id_str and int(float(bad_id_str)) in current_class_student_ids:
                        dislike_violations.append(f"({sid}, {int(float(bad_id_str))})")


        if dislike_violations:
            print(f"  !!! WARNING: Disliked pairs found in Classroom {c_id}: {', '.join(dislike_violations)}")


        print("-" * 29, end='\n\n')



else:
    print(f"\nNo solution found (Status: {solver.StatusName(status)})")
    print(f"  - Wall time: {solver.WallTime():.2f}s")
    print("Consider relaxing some constraints or increasing the time limit.")


Solution found (Status: OPTIMAL)
  - Objective value (Min/Max score difference): 0.0
  - Wall time: 12.98s

Total Students: 200, Total Classrooms: 6
Overall Average Student Score: 78.9300
Ideal Students per Class: 33.33
Ideal Total Score per Class: 2631.0000



:::Result:::
         id class
0    202505     1
1    202509     1
2    202512     1
3    202513     1
4    202519     1
..      ...   ...
195  202664     6
196  202673     6
197  202678     6
198  202686     6
199  202691     6

[200 rows x 2 columns]


-------- Classroom 0 --------
  Number of Students: 33
  Total Score: 2631
  Average Score: 79.73
  Leadership students: [202575, 202589, 202684, 202700]
  Count of club members: {'노래': 2, '댄스': 3, '야구': 4, '미술': 5, '밴드': 3, '축구': 4, '코딩': 2, '연극': 5, '봉사': 2, '독서': 3}
  Count of prev classes: {'a': 6, 'b': 6, 'c': 6, 'f': 5, 'd': 5, 'e': 5}
  Sex ratio: M 24 : 9 F
-----------------------------

-------- Classroom 1 --------
  Number of Students: 33
  Total Score: 2631
  Averag